# Building LLM applications: Notebook 01

## Initialize

In [13]:
import os
import dotenv

from langchain_ollama import ChatOllama
from langchain_core.prompts import PromptTemplate

In [11]:
MODEL = 'llama3.2'

In [12]:
# Read fro `.env` file
dotenv.load_dotenv()

OLLAMA_URL = os.getenv('OLLAMA_URL')
print(f"Using Ollama server: {OLLAMA_URL if OLLAMA_URL else 'local'}")

Using Ollama server: local


## Excercise 01: Connect to an LLM

Connect to an LLM and make a simple request (e..g translate a senetence)

```
Translate to French: I like coffee.
```

Take a look at the LLM parameters, and full response 

In [8]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

messages = "Translate to French: I like coffee."

# Invoke the LLM
ret = llm.invoke(messages)
print(ret.content)

Je aime le café.


In [9]:
# Show details of the response
print(ret.model_dump_json(indent=2))

{
  "content": "Je aime le café.",
  "additional_kwargs": {},
  "response_metadata": {
    "model": "llama3.2",
    "created_at": "2025-01-15T14:10:52.539095Z",
    "done": true,
    "done_reason": "stop",
    "total_duration": 209273167,
    "load_duration": 31747209,
    "prompt_eval_count": 33,
    "prompt_eval_duration": 98000000,
    "eval_count": 6,
    "eval_duration": 78000000,
    "message": {
      "role": "assistant",
      "content": "",
      "images": null,
      "tool_calls": null
    }
  },
  "type": "ai",
  "name": null,
  "id": "run-1316bc36-ec4a-4b68-b178-11c35fdb24dc-0",
  "example": false,
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 33,
    "output_tokens": 6,
    "total_tokens": 39
  }
}


## Excercise 02: Prompt template

In [ ]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable'
ret = runnable.invoke({"sentence": "I like coffee."})
print(ret.content)

Je aime le café.


## Excercise 03: Streaming

In [23]:
# Create the LLM object
llm = ChatOllama(model=MODEL, base_url=OLLAMA_URL)

# Create a prompt from a template
prompt_template = """
Translate to French:

{sentence}

"""

# Create a prompt object from the string template
prompt = PromptTemplate.from_template(prompt_template)


# Create a "chain" of runnable objects
runnable = prompt | llm

# Invoke the 'runnable' as a stream, print each block as it arrives
blocks = []
for b in runnable.stream({"sentence": "I like coffee."}):
    print(b.content, end='')
    blocks.append(b)

print(f"\n----\nReceived {len(blocks)} blocks")

J'aime le café.
----
Received 7 blocks


In [25]:
# What does a block look like
print(blocks[0])

content='J' additional_kwargs={} response_metadata={} id='run-48b32f94-9df8-4b97-99ff-24133c3b54b9'
